# Web Parsing

April 2019

In [1]:
import re
import pandas as pd
from bm_util import parse_table_from_url

### CoinMarketCap

Coins

In [2]:
coin = parse_table_from_url('https://coinmarketcap.com/all/views/all/')
coin = coin.iloc[:,1:6]
coin['CirculatingSupply'] = (pd.to_numeric(coin['CirculatingSupply']. \
                                      apply(lambda x: re.sub(r'[\W+]',"",x)), \
                                      errors='coerce').map('{:,.0f}'.format))
coin.head()

,Symbol,MarketCap,Price,CirculatingSupply,Volume24h
#,,,,,
1,BTC,"$88,705,338,047",$5031.10,"17,631,387","$16,780,175,632"
2,ETH,"$17,478,524,568",$165.59,"105,550,795","$7,140,141,428"
3,XRP,"$14,759,658,183",$0.353578,"41,743,765,071","$1,491,472,547"
4,LTC,"$5,582,401,999",$91.18,"61,225,436","$3,751,136,658"
5,BCH,"$5,354,332,584",$302.26,"17,714,188","$2,058,687,431"


Exchanges

In [5]:
exchange = parse_table_from_url('https://coinmarketcap.com/rankings/exchanges/')
exchange.drop('VolGraph7d',axis=1,inplace=True)
exchange.head()

,Name,AdjVol24h,Volume24h,Volume7d,Volume30d,NoMarkets,Change24h,Launched
#,,,,,,,,
1,DigiFinex,"$1,519,549,045","$1,519,549,045","$11,262,579,993","$26,270,306,073",164,-4.12%,Apr 2018
2,OKEx,"$1,486,671,821","$1,486,671,821","$12,213,214,294","$27,464,921,363",379,4.18%,Jan 2014
3,Binance,"$1,309,026,529","$1,309,026,529","$10,994,164,414","$32,059,480,103",463,-6.02%,Jul 2017
4,OOOBTC,"$1,304,190,515","$1,304,190,515","$2,621,679,312","$4,810,462,384",31,43.79%,Nov 2017
5,HitBTC,"$1,290,567,105","$1,290,567,105","$7,443,271,121","$21,572,556,044",790,-10.09%,Feb 2014
